In [47]:
import gzip
import json
import tensorflow as tf
import collections

tf.logging.set_verbosity(tf.logging.INFO)

In [48]:
#DATA_DIR = "data/v1.0/dev/nq-dev-0*.jsonl.gz"
DATA_DIR = "data/tiny-dev/nq-dev-sample.jsonl.gz"
max_position = 50
skip_nested_contexts = False
TextSpan = collections.namedtuple("TextSpan", "token_positions text")

In [49]:

examples = []
with gzip.GzipFile(DATA_DIR) as f:
    for line in f:
        example = json.loads(line,'utf-8')
        #print example['example_id']
        a = example['annotations']
        #long_answers = [ a['long_answer'] for a in example['annotations'] if a['long_answer']['start_byte'] >= 0]
        print a
        '''
        long_answer_bounds = [
          (la['start_byte'], la['end_byte']) for la in long_answers
          ]
        #print long_answer_bounds
        long_answer_counts = [
          long_answer_bounds.count(la) for la in long_answer_bounds
          ]
        '''
        print(example["long_answer_candidates"])
        #print long_answer_counts
        #print example.keys()
        #print "short_answers", a["short_answers"]
        #if len(a["short_answers"]) > 1 and len(a["long_answer"]) > 1:
        #    print (a)
        ''' if len(a["short_answers"]) > 1:
            print long_answers
            print long_answer_counts
            print "short_answers", a["short_answers"]
        '''
        examples.append(example)
        break
        

[{u'long_answer': {u'end_token': -1, u'candidate_index': -1, u'start_byte': -1, u'end_byte': -1, u'start_token': -1}, u'yes_no_answer': u'NONE', u'annotation_id': 4323936797498927989, u'short_answers': []}, {u'long_answer': {u'end_token': -1, u'candidate_index': -1, u'start_byte': -1, u'end_byte': -1, u'start_token': -1}, u'yes_no_answer': u'NONE', u'annotation_id': 13037645000009169623L, u'short_answers': []}, {u'long_answer': {u'end_token': -1, u'candidate_index': -1, u'start_byte': -1, u'end_byte': -1, u'start_token': -1}, u'yes_no_answer': u'NONE', u'annotation_id': 4439059471919323171, u'short_answers': []}, {u'long_answer': {u'end_token': 161, u'candidate_index': 0, u'start_byte': 43178, u'end_byte': 44666, u'start_token': 44}, u'yes_no_answer': u'NONE', u'annotation_id': 15051359051424338858L, u'short_answers': [{u'end_token': 159, u'start_token': 140, u'start_byte': 44318, u'end_byte': 44657}]}, {u'long_answer': {u'end_token': 161, u'candidate_index': 0, u'start_byte': 43178, u

In [94]:
CANDIDATES_COUNT = {"Paragraph": 0, "Table" : 0, "List" : 0, "Other" : 0}
def get_text_span(example, span):
    """Returns the text in the example's document in the given token span."""
    token_positions = []
    tokens = []
    for i in range(span["start_token"], span["end_token"]):
        t = example["document_tokens"][i]
        if not t["html_token"]:
            token_positions.append(i)
            token = t["token"].replace(" ", "")
            tokens.append(token)
    return TextSpan(token_positions, " ".join(tokens))

def get_candidate_text(e, idx):
    """Returns a text representation of the candidate at the given index."""
    # No candidate at this index.
    if idx < 0 or idx >= len(e["long_answer_candidates"]):
        return TextSpan([], "")
    # This returns an actual candidate.
    return get_text_span(e, e["long_answer_candidates"][idx])

def should_skip_context(e, idx):
    t = get_candidate_type(e, idx)
    CANDIDATES_COUNT[t] += 1
    if (skip_nested_contexts and
        not e["long_answer_candidates"][idx]["top_level"]):
        return True
    elif not get_candidate_text(e, idx).text.strip():
        # Skip empty contexts.
        return True
    else:
        return False

def candidates_iter(e):
    """Yield's the candidates that should not be skipped in an example."""
    for idx, c in enumerate(e["long_answer_candidates"]):
        if should_skip_context(e, idx):
            continue
    yield idx, c

def get_candidate_type(e, idx):
    """Returns the candidate's type: Table, Paragraph, List or Other."""
    c = e["long_answer_candidates"][idx]
    first_token = e["document_tokens"][c["start_token"]]["token"]
    if first_token == "<Table>":
        return "Table"
    elif first_token == "<P>":
        return "Paragraph"
    elif first_token in ("<Ul>", "<Dl>", "<Ol>"):
        return "List"
    elif first_token in ("<Tr>", "<Li>", "<Dd>", "<Dt>"):
        return "Other"
    else:
        tf.logging.warning("Unknoww candidate type found: %s", first_token)
    return "Other"

def add_candidate_types_and_positions(e):
    """Adds type and position info to each candidate in the document."""
    counts = collections.defaultdict(int)
    for idx, c in candidates_iter(e):
        context_type = get_candidate_type(e, idx)
        if counts[context_type] < max_position:
            counts[context_type] += 1
            c["type_and_position"] = "[%s=%d]" % (context_type, counts[context_type])

def load_example(line):
    e = json.loads(line, object_pairs_hook=collections.OrderedDict)
    cand_idx = (e["annotations"][0]["long_answer"]["candidate_index"])
    if cand_idx != -1:
        print(e["annotations"][0]["long_answer"])
        print(e["long_answer_candidates"])
        print(get_candidate_type(e, cand_idx))
        
    #add_candidate_types_and_positions(e)
    
def read_nq_examples(input_file, is_training):
    """Read a NQ json file into a list of NqExample."""
    input_paths = tf.gfile.Glob(input_file)
    input_data = []

    def _open(path):
        if path.endswith(".gz"):
            return gzip.GzipFile(fileobj=tf.gfile.Open(path, "r"))
        else:
            return tf.gfile.Open(path, "r")

    for path in input_paths:
        tf.logging.info("Reading: %s", path)
        with _open(path) as input_file:
            for line in input_file:
                load_example(line)
            break

In [95]:
read_nq_examples(DATA_DIR, False)
tf.logging.info("Disrtribution of long answer types: %s",CANDIDATES_COUNT)

I0917 20:26:07.852518 140638234494784 <ipython-input-94-3dae14726c53>:87] Reading: data/tiny-dev/nq-dev-sample.jsonl.gz


OrderedDict([(u'candidate_index', 2), (u'end_byte', 342045), (u'end_token', 6124), (u'start_byte', 39414), (u'start_token', 394)])
[OrderedDict([(u'end_byte', 36110), (u'end_token', 179), (u'start_byte', 34485), (u'start_token', 41), (u'top_level', True)]), OrderedDict([(u'end_byte', 39086), (u'end_token', 388), (u'start_byte', 37786), (u'start_token', 233), (u'top_level', True)]), OrderedDict([(u'end_byte', 342045), (u'end_token', 6124), (u'start_byte', 39414), (u'start_token', 394), (u'top_level', True)]), OrderedDict([(u'end_byte', 40197), (u'end_token', 409), (u'start_byte', 39475), (u'start_token', 395), (u'top_level', False)]), OrderedDict([(u'end_byte', 41751), (u'end_token', 448), (u'start_byte', 40213), (u'start_token', 409), (u'top_level', False)]), OrderedDict([(u'end_byte', 43338), (u'end_token', 485), (u'start_byte', 41752), (u'start_token', 448), (u'top_level', False)]), OrderedDict([(u'end_byte', 44484), (u'end_token', 496), (u'start_byte', 43339), (u'start_token', 485),

OrderedDict([(u'candidate_index', 28), (u'end_byte', 62871), (u'end_token', 547), (u'start_byte', 62733), (u'start_token', 518)])
[OrderedDict([(u'end_byte', 60014), (u'end_token', 272), (u'start_byte', 55151), (u'start_token', 14), (u'top_level', True)]), OrderedDict([(u'end_byte', 55373), (u'end_token', 21), (u'start_byte', 55223), (u'start_token', 15), (u'top_level', False)]), OrderedDict([(u'end_byte', 55784), (u'end_token', 27), (u'start_byte', 55374), (u'start_token', 21), (u'top_level', False)]), OrderedDict([(u'end_byte', 55975), (u'end_token', 37), (u'start_byte', 55785), (u'start_token', 27), (u'top_level', False)]), OrderedDict([(u'end_byte', 56379), (u'end_token', 60), (u'start_byte', 55976), (u'start_token', 37), (u'top_level', False)]), OrderedDict([(u'end_byte', 56360), (u'end_token', 58), (u'start_byte', 56092), (u'start_token', 43), (u'top_level', False)]), OrderedDict([(u'end_byte', 56354), (u'end_token', 57), (u'start_byte', 56255), (u'start_token', 52), (u'top_level

OrderedDict([(u'candidate_index', 14), (u'end_byte', 66837), (u'end_token', 752), (u'start_byte', 63051), (u'start_token', 474)])
[OrderedDict([(u'end_byte', 59449), (u'end_token', 149), (u'start_byte', 55487), (u'start_token', 14), (u'top_level', True)]), OrderedDict([(u'end_byte', 55647), (u'end_token', 21), (u'start_byte', 55543), (u'start_token', 15), (u'top_level', False)]), OrderedDict([(u'end_byte', 57233), (u'end_token', 33), (u'start_byte', 55648), (u'start_token', 21), (u'top_level', False)]), OrderedDict([(u'end_byte', 57322), (u'end_token', 42), (u'start_byte', 57234), (u'start_token', 33), (u'top_level', False)]), OrderedDict([(u'end_byte', 57561), (u'end_token', 65), (u'start_byte', 57323), (u'start_token', 42), (u'top_level', False)]), OrderedDict([(u'end_byte', 57790), (u'end_token', 75), (u'start_byte', 57562), (u'start_token', 65), (u'top_level', False)]), OrderedDict([(u'end_byte', 57886), (u'end_token', 87), (u'start_byte', 57791), (u'start_token', 75), (u'top_level

OrderedDict([(u'candidate_index', 0), (u'end_byte', 52280), (u'end_token', 179), (u'start_byte', 41470), (u'start_token', 23)])
[OrderedDict([(u'end_byte', 52280), (u'end_token', 179), (u'start_byte', 41470), (u'start_token', 23), (u'top_level', True)]), OrderedDict([(u'end_byte', 42181), (u'end_token', 40), (u'start_byte', 42064), (u'start_token', 29), (u'top_level', False)]), OrderedDict([(u'end_byte', 42307), (u'end_token', 52), (u'start_byte', 42182), (u'start_token', 40), (u'top_level', False)]), OrderedDict([(u'end_byte', 42545), (u'end_token', 67), (u'start_byte', 42308), (u'start_token', 52), (u'top_level', False)]), OrderedDict([(u'end_byte', 42731), (u'end_token', 76), (u'start_byte', 42546), (u'start_token', 67), (u'top_level', False)]), OrderedDict([(u'end_byte', 42950), (u'end_token', 85), (u'start_byte', 42732), (u'start_token', 76), (u'top_level', False)]), OrderedDict([(u'end_byte', 43128), (u'end_token', 100), (u'start_byte', 42951), (u'start_token', 85), (u'top_level'

...

OrderedDict([(u'candidate_index', 0), (u'end_byte', 43543), (u'end_token', 179), (u'start_byte', 41563), (u'start_token', 22)])
[OrderedDict([(u'end_byte', 43543), (u'end_token', 179), (u'start_byte', 41563), (u'start_token', 22), (u'top_level', True)]), OrderedDict([(u'end_byte', 44436), (u'end_token', 287), (u'start_byte', 43544), (u'start_token', 179), (u'top_level', True)]), OrderedDict([(u'end_byte', 46627), (u'end_token', 480), (u'start_byte', 44437), (u'start_token', 287), (u'top_level', True)]), OrderedDict([(u'end_byte', 47073), (u'end_token', 497), (u'start_byte', 46628), (u'start_token', 480), (u'top_level', True)]), OrderedDict([(u'end_byte', 47238), (u'end_token', 513), (u'start_byte', 47074), (u'start_token', 497), (u'top_level', True)]), OrderedDict([(u'end_byte', 54305), (u'end_token', 837), (u'start_byte', 53751), (u'start_token', 752), (u'top_level', True)]), OrderedDict([(u'end_byte', 54961), (u'end_token', 942), (u'start_byte', 54306), (u'start_token', 837), (u'top_

OrderedDict([(u'candidate_index', 6), (u'end_byte', 66323), (u'end_token', 664), (u'start_byte', 65632), (u'start_token', 606)])
[OrderedDict([(u'end_byte', 59043), (u'end_token', 173), (u'start_byte', 57849), (u'start_token', 65), (u'top_level', True)]), OrderedDict([(u'end_byte', 60183), (u'end_token', 276), (u'start_byte', 59044), (u'start_token', 173), (u'top_level', True)]), OrderedDict([(u'end_byte', 63799), (u'end_token', 410), (u'start_byte', 63325), (u'start_token', 382), (u'top_level', True)]), OrderedDict([(u'end_byte', 64214), (u'end_token', 439), (u'start_byte', 63800), (u'start_token', 410), (u'top_level', True)]), OrderedDict([(u'end_byte', 64717), (u'end_token', 502), (u'start_byte', 64215), (u'start_token', 439), (u'top_level', True)]), OrderedDict([(u'end_byte', 65631), (u'end_token', 606), (u'start_byte', 65029), (u'start_token', 508), (u'top_level', True)]), OrderedDict([(u'end_byte', 66323), (u'end_token', 664), (u'start_byte', 65632), (u'start_token', 606), (u'top

OrderedDict([(u'candidate_index', 0), (u'end_byte', 67783), (u'end_token', 142), (u'start_byte', 66268), (u'start_token', 42)])
[OrderedDict([(u'end_byte', 67783), (u'end_token', 142), (u'start_byte', 66268), (u'start_token', 42), (u'top_level', True)]), OrderedDict([(u'end_byte', 68323), (u'end_token', 177), (u'start_byte', 67784), (u'start_token', 142), (u'top_level', True)]), OrderedDict([(u'end_byte', 70835), (u'end_token', 235), (u'start_byte', 70723), (u'start_token', 230), (u'top_level', True)]), OrderedDict([(u'end_byte', 71681), (u'end_token', 355), (u'start_byte', 70836), (u'start_token', 235), (u'top_level', True)]), OrderedDict([(u'end_byte', 70950), (u'end_token', 246), (u'start_byte', 70841), (u'start_token', 236), (u'top_level', False)]), OrderedDict([(u'end_byte', 71167), (u'end_token', 281), (u'start_byte', 70951), (u'start_token', 246), (u'top_level', False)]), OrderedDict([(u'end_byte', 71389), (u'end_token', 322), (u'start_byte', 71168), (u'start_token', 281), (u'to

OrderedDict([(u'candidate_index', 16), (u'end_byte', 47642), (u'end_token', 270), (u'start_byte', 46416), (u'start_token', 207)])
[OrderedDict([(u'end_byte', 43279), (u'end_token', 75), (u'start_byte', 41787), (u'start_token', 24), (u'top_level', True)]), OrderedDict([(u'end_byte', 43262), (u'end_token', 74), (u'start_byte', 41887), (u'start_token', 25), (u'top_level', False)]), OrderedDict([(u'end_byte', 46415), (u'end_token', 207), (u'start_byte', 43280), (u'start_token', 75), (u'top_level', True)]), OrderedDict([(u'end_byte', 43591), (u'end_token', 87), (u'start_byte', 43389), (u'start_token', 78), (u'top_level', False)]), OrderedDict([(u'end_byte', 44671), (u'end_token', 107), (u'start_byte', 44187), (u'start_token', 91), (u'top_level', False)]), OrderedDict([(u'end_byte', 44763), (u'end_token', 112), (u'start_byte', 44672), (u'start_token', 107), (u'top_level', False)]), OrderedDict([(u'end_byte', 45126), (u'end_token', 131), (u'start_byte', 44764), (u'start_token', 112), (u'top_l

I0917 20:26:53.244371 140638234494784 <ipython-input-95-78b894cdb3e2>:2] Disrtribution of long answer types: {'Other': 0, 'Table': 0, 'Paragraph': 0, 'List': 0}
